In [102]:
from json_logic import jsonLogic
from json_logic.builtins import BUILTINS

In [103]:

jsonLogic({"==": [1, 1]})
jsonLogic({"==": [{"var": "a"}, {"var": "b"}]}, {"a": 1, "b": 1})
jsonLogic(
    {"in": [
        {"code": "CPSC", "number": "526"}, 
        {"var": "courses"}
    ]},
    {"courses": [{"code": "CPSC", "number": "526"}]},
)

True

In [104]:
def _check_course_number(number_rule: str, number_data: str):
    # For rules like "400+"
    if number_rule.endswith("+"):
        number_rule_actual = int(number_rule[:-1])
        number_data_actual = int(number_data)
        return number_data_actual >= number_rule_actual
    
    number_rule_actual = int(number_rule)
    number_data_actual = int(number_data)
    return number_data_actual == number_rule_actual

def _find_taken_courses(taken_courses: list, code: str, number: str):
    for course in taken_courses:
        if course["code"] == code and _check_course_number(number, course["number"]):
            return course
    return None

def taken(data: dict, code, number):
    courses = data.get("courses", [])

    if _find_taken_courses(courses, code, number):
        return True
    return False

def units(data: dict, config: dict):
    # Rule
    required_units = config.get("required", 0)
    from_courses = config.get("from", [])

    # Data
    courses = data.get("courses", [])

    taken_units = 0
    for code, number in from_courses:
        course = _find_taken_courses(courses, code, number)
        if course:
            taken_units += course["units"]

    return taken_units >= required_units

def consent(data: dict, type: str, code: str):
    return True

ops = {**BUILTINS, "taken": taken, "units": units, "consent": consent}

In [105]:
# CPSC 219: CPSC 217 or DATA 211.
rule = {
    "or": [
        {"taken": ["CPSC", "217"]},
        {"taken": ["DATA", "211"]}
    ]
}
data = {"courses": [{"code": "CPSC", "number": "217", "units": 3}]}
jsonLogic(rule, data, ops)

True

In [106]:
# CPSC 313: MATH 271 or 273; and PHIL 279 or 377; and 3 units from CPSC 219, 233 or 235.
rule = {
    "and": [
        {"or": [{"taken": ["MATH", "271"]}, {"taken": ["MATH", "273"]}]},
        {"or": [{"taken": ["PHIL", "279"]}, {"taken": ["PHIL", "377"]}]},
        {
            "units": {
                "required": 3,
                "from": [
                    ["CPSC", "219"],
                    ["CPSC", "233"],
                    ["CPSC", "235"]
                ]
            }
        },
    ]
}
data = {
    "courses": [
        {"code": "MATH", "number": "271", "units": 3},
        {"code": "PHIL", "number": "279", "units": 3},
        {"code": "CPSC", "number": "219", "units": 3},
    ]
}
jsonLogic(rule, data, ops)

True

In [107]:
# CPSC 359: CPSC 355 and PHIL 279 or 377.
rule = {
    "and": [
        {"taken": ["CPSC", "355"]},
        {"or": [{"taken": ["PHIL", "279"]}, {"taken": ["PHIL", "377"]}]},
    ]
}
data = {
    "courses": [
        {"code": "CPSC", "number": "355", "units": 3},
        {"code": "PHIL", "number": "279", "units": 3},
    ]
}
jsonLogic(rule, data, ops)

True

In [108]:
# DATA 433: CPSC 217, MATH 267 and 311.
rule = {
    "and": [
        {"taken": ["CPSC", "217"]},
        {"taken": ["MATH", "267"]},
        {"taken": ["MATH", "311"]},
    ]
}
data = {
    "courses": [
        {"code": "CPSC", "number": "217", "units": 3},
        {"code": "MATH", "number": "267", "units": 3},
        {"code": "MATH", "number": "311", "units": 3},
    ]
}
jsonLogic(rule, data, ops)

True

In [109]:
# CPSC 233: CPSC 231 and admission to Computer Science, Bioinformatics, or Natural Science with a primary concentration in Computer Science.
rule = {
    "and": [
        {"taken": ["CPSC", "231"]},
    ]
}
data = {
    "courses": [
        {"code": "CPSC", "number": "231", "units": 3},
    ]
}
jsonLogic(rule, data, ops)

True

In [110]:
# CPSC 399: Consent of the Department.
rule = {
    "consent": ["Department", "CPSC"]
}
data = {}
jsonLogic(rule, data, ops)

True

In [111]:
# DATA 599: Consent of the Faculty.
rule = {
    "consent": ["Faculty", "SC"]
}
data = {}
jsonLogic(rule, data, ops)

True